In [ ]:
# Import
import os
import pandas as pd
import numpy as np

from pathlib import Path

from CellPacking.tissuegeneration import sheet_init
from CellPacking.dynamics import Compression, AnisotropicLineTension

from tyssue import Sheet
from tyssue import PlanarGeometry
from tyssue.solvers import QSSolver
from tyssue.solvers.viscous import EulerSolver
from tyssue.behaviors.event_manager import EventManager
from tyssue.behaviors.sheet.basic_events import reconnect
from tyssue.dynamics import model_factory, effectors
from tyssue.core.history import HistoryHdf5 


import matplotlib.pyplot as plt
from tyssue.draw import sheet_view
from CellPacking.plot import superimpose_sheet_view
from CellPacking.plot import sheet_view as ply_sheet_view

In [ ]:
SIM_DIR = Path('/mnt/sda1/Sophie/Simulations/20220905_2D_QS')
save_dir = SIM_DIR
# sim_save_dir = SIM_DIR/f'simu_proteins'
try:
    os.mkdir(save_dir)
except FileExistsError:
    pass

# Apical sheet init

In [ ]:
repeat = np.arange(4)
gammas = np.linspace(0, 0.2, 21)

# Number of cells in x and y axis
nx = 40
ny = 40
noise = 0.5
phi = np.pi/2

for r in repeat:
    sim_save_dir = SIM_DIR/str(r)
    try:
        os.mkdir(sim_save_dir)
    except FileExistsError:
        pass
    for g in gammas : 
        gamma_0 = g
        
        attempts=0
        while attempts < 5:
            try : 
                apical_sheet, geom = sheet_init(nx, ny, gamma_0, phi, noise)
                break
            except:
                attempts +=1

        print('number of attempts: ',attempts)

        apical_sheet.get_opposite()

        apical_sheet.update_specs({"settings":{"dt":0.01,
                                             'threshold_length': 0.01,
                                             'p_4': 0.1,
                                             'p_5p': 0.01,}})

        apical_sheet.face_df["analyse_cell"]=False
        apical_sheet.face_df.loc[apical_sheet.face_df[(apical_sheet.face_df.x>-10) & 
                                              (apical_sheet.face_df.x<10) &
                                             (apical_sheet.face_df.y>-10) & 
                                              (apical_sheet.face_df.y<10)].index, "analyse_cell"] = True
        
        apical_sheet.face_df['prefered_area'] = 1.
        apical_sheet.face_df['area_elasticity'] = 1.
        apical_sheet.face_df['prefered_perimeter'] = 3.*np.sqrt(1)
        apical_sheet.face_df['perimeter_elasticity'] = 1.
        apical_sheet.vert_df['viscosity'] = 1.0
        apical_sheet.edge_df['is_active'] = 1
        

        apical_sheet.face_df['id_'] = apical_sheet.face_df.index
        apical_sheet.edge_df['id_'] = apical_sheet.edge_df.index
        apical_sheet.vert_df['id_'] = apical_sheet.vert_df.index

        # Solver
        solver_qs = QSSolver(with_t1=False, with_t3=False, with_collisions=False)

        # Manager
        manager = EventManager('face')
        manager.append(reconnect)


        # Model
        model = model_factory(
            [
        #         AnisotropicLineTension, 
                effectors.FaceAreaElasticity,
                effectors.PerimeterElasticity,
            ], effectors.FaceAreaElasticity)

        for i in range (10):
            manager.execute(apical_sheet)
            res = solver_qs.find_energy_min(apical_sheet, geom, model, periodic=False, options={"gtol": 1e-8})
            if res.success is False:
                print (res)
            manager.update()

        # Model
        model = model_factory(
            [
                AnisotropicLineTension,
                effectors.FaceAreaElasticity,
                effectors.PerimeterElasticity,
            ], effectors.FaceAreaElasticity)
        # Manager
        manager = EventManager('face', )#track_event=True)
        manager.append(reconnect)
        # History
        history = HistoryHdf5(apical_sheet, 
                              hf5file=os.path.join(sim_save_dir, 'test_'+str(g)+'.hf5'),
                             )#track_event=True)

#         # Solver
#         solver = EulerSolver(apical_sheet,
#                              geom,
#                              model,
#                              history=history,
#                              manager=manager,
#                              auto_reconnect=True)

#         solver.solve(tf=20,
#                      dt=apical_sheet.settings['dt'],

#                     )
        
        for i in range (50):
            manager.execute(history.sheet)
            res = solver_qs.find_energy_min(history.sheet, geom, model, periodic=False, options={"gtol": 1e-8})
            if res.success is False:
                print (res)
            manager.update()
            
            history.record(time_stamp=i)


In [ ]:
result = pd.DataFrame(columns = ['repeat', 'gamma', 'nb_change', 'tot_cell'])

repeat = np.arange(4)
gammas = np.linspace(0, 0.2, 21)

for r in repeat:
    sim_save_dir = SIM_DIR/str(r)


    for g in gammas : 
        history = HistoryHdf5.from_archive(os.path.join(sim_save_dir, 'test_'+str(g)+'.hf5'))
        unique = np.unique(history.retrieve(0).face_df.num_sides - history.retrieve(20).face_df.num_sides,
                           return_counts=True)
        id_delete = np.where(unique[0]==0)[0]
        unique = (np.delete(unique[0], id_delete), np.delete(unique[1], id_delete))

        result = pd.concat([result, pd.DataFrame({'repeat':r,
                                                  'gamma':g, 
                                                  'nb_change':np.sum(unique[1]),
                                                 'tot_cell': history.retrieve(0).Nf},
                          index=[0])],
                          ignore_index=True)

#         fig, ax = sheet_view(history.retrieve(0))
#         fig.set_size_inches((10,10))
#         fig.savefig('test_'+str(g)+'_init.png')

#         fig, ax = sheet_view(history.retrieve(20))
#         fig.set_size_inches((10,10))
#         fig.savefig('test_'+str(g)+'_end.png')



In [ ]:
result.to_csv('20220812_results.csv')

In [ ]:
fig, ax = plt.subplots()
ax.plot(result.gamma, result.nb_change/2, '.', markersize=10)
ax.set_xlabel('gamma')
ax.set_ylabel('number of cell that have neighbouring change')

fig.set_size_inches((10,10))

In [ ]:
g=0.1
history = HistoryHdf5.from_archive(os.path.join(sim_save_dir, 'test_'+str(g)+'.hf5'))

In [ ]:
np.unique(history.retrieve(0).face_df.num_sides - history.retrieve(20).face_df.num_sides, return_counts=True)

In [ ]:
apical_sheet_init = history.retrieve(0)
apical_sheet = history.retrieve(20)

apical_sheet.face_df['color'] = 0
nb_change = apical_sheet_init.face_df.num_sides - apical_sheet.face_df.num_sides
apical_sheet.face_df.loc[nb_change[nb_change==1].index.to_numpy(), 'color'] = 1
apical_sheet.face_df.loc[nb_change[nb_change>2].index.to_numpy(), 'color'] = 2
apical_sheet.face_df.loc[nb_change[nb_change==-1].index.to_numpy(), 'color'] = -1
apical_sheet.face_df.loc[nb_change[nb_change<-2].index.to_numpy(), 'color'] = -2


import plotly.express as px
import plotly.graph_objects as go
import itertools
from tyssue.core.objects import _ordered_edges
from tyssue.draw.plt_draw import _get_lines


a_lines_x, a_lines_y = _get_lines(apical_sheet_init, list('xy'))

b_lines_x, b_lines_y = _get_lines(apical_sheet, list('xy'))

df = pd.DataFrame([np.concatenate((a_lines_x, b_lines_x)),
                   np.concatenate((a_lines_y, b_lines_y)), 
                   np.concatenate((np.repeat('start', len(a_lines_x)), np.repeat('end', len(b_lines_x))))])
df = df.T
df.columns = ['lines_x', 'lines_y', 'tyssue']

fig = px.line(df, x='lines_x', y='lines_y', color='tyssue')


# 2 changes or more
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']>=2].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#EEA000',  #dark orange
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))

# # 1 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==1].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#EEE60C',  #light orange
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))

# # -1 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==-1].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#7CB5EF',  # blue 
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))
    
# # -2 change
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']==-2].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#0566CA',  # dark blue 
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))


fig.update_layout(
    xaxis_range=[-10,10],
    yaxis_range=[-10,10],
    autosize=False,
    width=1000,
    height=1000,
)

fig.show()

In [ ]:
remove_vert = []
add_vert = []

for i, event in history.trackevent.items():
    if i!=0:
        if event['from_vert_r'] is not None :
            remove_vert.append(event['from_vert_r'])
        add_vert.append(event['from_vert'])
        
remove_vert = [item for sublist in remove_vert for item in sublist]
add_vert = [item for sublist in add_vert for item in sublist]

np.intersect1d(add_vert, remove_vert)

In [ ]:
apical_sheet.face_df['color'] = 0
apical_sheet.face_df.loc[apical_sheet.edge_df[apical_sheet.edge_df['srce'].isin(np.intersect1d(add_vert, remove_vert))]['face'].to_numpy(),
                        'color'] = 5

apical_sheet.face_df.loc[apical_sheet.edge_df[apical_sheet.edge_df['trgt'].isin(np.intersect1d(add_vert, remove_vert))]['face'].to_numpy(),
                        'color'] = 5


import plotly.express as px
import plotly.graph_objects as go
import itertools
from tyssue.core.objects import _ordered_edges
from tyssue.draw.plt_draw import _get_lines


a_lines_x, a_lines_y = _get_lines(apical_sheet_init, list('xy'))

b_lines_x, b_lines_y = _get_lines(apical_sheet, list('xy'))

df = pd.DataFrame([np.concatenate((a_lines_x, b_lines_x)),
                   np.concatenate((a_lines_y, b_lines_y)), 
                   np.concatenate((np.repeat('start', len(a_lines_x)), np.repeat('end', len(b_lines_x))))])
df = df.T
df.columns = ['lines_x', 'lines_y', 'tyssue']

fig = px.line(df, x='lines_x', y='lines_y', color='tyssue')


# 2 changes or more
color_cells = apical_sheet.face_df[apical_sheet.face_df['color']>=2].index
apical_sheet.reset_index()
for color_cell in color_cells:
    edges = list(itertools.chain(*_ordered_edges(apical_sheet.edge_df[apical_sheet.edge_df['face']==color_cell])))[0::3]
    fig.add_trace(go.Scatter(x=apical_sheet.vert_df.loc[edges]['x'].to_numpy(),
                             y=apical_sheet.vert_df.loc[edges]['y'].to_numpy(),
                            fill='toself',
                             fillcolor='#EEA000',  #dark orange
                             line=dict(width=0),
                             showlegend=True,
                             name = color_cell,
                             mode='lines'))
fig.update_layout(
    xaxis_range=[-10,10],
    yaxis_range=[-10,10],
    autosize=False,
    width=1000,
    height=1000,
)

fig.show()